#### Spark UI

how to access the spark UI

YARN Resource manager and Spark UI

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
builder. \
appName("caching session demo4"). \
config("spark.sql.warehouse.dir","/user/itv020649/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [3]:
spark

#### Cashe and Persist

spark is already in memory than why cache()?

df1 = read a order file from disk -- if i fee this df1 will be reused again and again we can cache it.

df2 = df1.xfr1  df3 = df2.xfr2

df3.show() --- all stages will run from begning from read.

this is solved in cache() (it is lazy)

should not cache() large DF or it can make issue

RDD - cache in memory

DF or sql table - it cache in memory and disk

10 GB file- 128 block size- 80 blocks-not enough memrory (only 50 are in memory)-remaining 50 in disk- this disk part is skipped in RDD

Disk has 2 parts - worker node-hdfs / local disk(little faster)

persist- memory and disk as option

cache- memory+disk by default

### Cache praticals

In [4]:
orders_schema = 'order_id long,order_date date,cust_id long,order_status string'

In [21]:
raw_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [22]:
raw_df.show()

+--------+----------+-------+---------------+
|order_id|order_date|cust_id|   order_status|
+--------+----------+-------+---------------+
|       1|2013-07-25|  11599|         CLOSED|
|       2|2013-07-25|    256|PENDING_PAYMENT|
|       3|2013-07-25|  12111|       COMPLETE|
|       4|2013-07-25|   8827|         CLOSED|
|       5|2013-07-25|  11318|       COMPLETE|
|       6|2013-07-25|   7130|       COMPLETE|
|       7|2013-07-25|   4530|       COMPLETE|
|       8|2013-07-25|   2911|     PROCESSING|
|       9|2013-07-25|   5657|PENDING_PAYMENT|
|      10|2013-07-25|   5648|PENDING_PAYMENT|
|      11|2013-07-25|    918| PAYMENT_REVIEW|
|      12|2013-07-25|   1837|         CLOSED|
|      13|2013-07-25|   9149|PENDING_PAYMENT|
|      14|2013-07-25|   9842|     PROCESSING|
|      15|2013-07-25|   2568|       COMPLETE|
|      16|2013-07-25|   7276|PENDING_PAYMENT|
|      17|2013-07-25|   2667|       COMPLETE|
|      18|2013-07-25|   1205|         CLOSED|
|      19|2013-07-25|   9488|PENDI

In [7]:
raw_df.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- order_date: date (nullable = true)
 |-- cust_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [8]:
raw_df.count()

25831125

1.1gb

9 blocks in hdfs

count

9 partition in your data frame

9 task
task1 -> 1000
task2 -> 2000
task3 -> 1500

1 from each executor and sum

In [9]:
order_df_cached = raw_df.cache()

In [10]:
order_df_cached.show()

+--------+----------+-------+---------------+
|order_id|order_date|cust_id|   order_status|
+--------+----------+-------+---------------+
|       1|2013-07-25|  11599|         CLOSED|
|       2|2013-07-25|    256|PENDING_PAYMENT|
|       3|2013-07-25|  12111|       COMPLETE|
|       4|2013-07-25|   8827|         CLOSED|
|       5|2013-07-25|  11318|       COMPLETE|
|       6|2013-07-25|   7130|       COMPLETE|
|       7|2013-07-25|   4530|       COMPLETE|
|       8|2013-07-25|   2911|     PROCESSING|
|       9|2013-07-25|   5657|PENDING_PAYMENT|
|      10|2013-07-25|   5648|PENDING_PAYMENT|
|      11|2013-07-25|    918| PAYMENT_REVIEW|
|      12|2013-07-25|   1837|         CLOSED|
|      13|2013-07-25|   9149|PENDING_PAYMENT|
|      14|2013-07-25|   9842|     PROCESSING|
|      15|2013-07-25|   2568|       COMPLETE|
|      16|2013-07-25|   7276|PENDING_PAYMENT|
|      17|2013-07-25|   2667|       COMPLETE|
|      18|2013-07-25|   1205|         CLOSED|
|      19|2013-07-25|   9488|PENDI

for show it only cached 1st partition 11% same for head-- check storage

for tail it will cache will be cached

In [11]:
order_df_cached.tail(10)

AttributeError: 'DataFrame' object has no attribute 'tail'

Memory Deserilized- object form like formal data,takes more space

serialized - binary format(more optimizedterm of space, but convertion it takes extra cpu cycle)

in-memory-> deserilized
disk-> serialized

caching:
in-memory-> deserilized/serialized
disk-> serialized

In [23]:
order_df_cached.count()
#it will deal with all records so all partition will be cached

25831125

dynamic resource allocation is enabled, min two executor and max 10

id application is ideal the extra executor will be removed but not less then min

node_loacl -> normal/disk

processed local -> cache

In [ ]:
order_df_cached.unpersist()

### More on CACHE

In [ ]:
raw_df.cache()

In [24]:
filtered_results = raw_df.filter("cust_id == 1000000")


In [25]:
filtered_results.show()
# it will cache everything

+--------+----------+-------+------------+
|order_id|order_date|cust_id|order_status|
+--------+----------+-------+------------+
+--------+----------+-------+------------+



In [26]:
raw_df.select("order_id","order_status").filter("order_status=='CLOSED'").cache()

order_id,order_status
1,CLOSED
4,CLOSED
12,CLOSED
18,CLOSED
24,CLOSED
25,CLOSED
37,CLOSED
51,CLOSED
57,CLOSED
61,CLOSED


In [27]:
raw_df.select("order_id","order_status").filter("order_status=='CLOSED'").cache().count()
#count will cache everything
#if rerun it will hit cache

2833500

In [28]:
raw_df.filter("order_status=='CLOSED'").select("order_id","order_status").cache().count()
#even the query is samemoving filters ahead so it doesnt hit chache
#cache only checks analysised plan not optimised and if it matches its hit cache


2833500

In [29]:
raw_df.select("order_id").filter("order_status=='CLOSED'").cache().count()
#it will hit cache

2833500

In [30]:
raw_df.select("order_id").filter("order_status=='CLOSED'").cache().count()
#it will not hit cache

2833500

### Cache - InMemory Table Cache | Node Local & Process Local

In [31]:
raw_df.count()

25831125

In [32]:
raw_df.distinct().count()

68883

1.1gb 9blocks 9partition 9tasks 

e1-128mb-25lac record-distinct-68883

e2-128mb-25lac record
.
.
e9-128mb-25lac record

wheenever we invoke wide xfr like distinct 200 partition(default) the results will go to 200 partition

9 task->200task->1 task(aggreated)->count o/p

if perticular the record went to one partition next time it will go to same partition, hash funtion should be consistent

In [33]:
cache_df = raw_df.cache()
#why to store in another DF

In [34]:
cache_df.count()

25831125

In [35]:
cache_df.unpersist()

order_id,order_date,cust_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


### Caching Spark table

In [36]:
raw_df.show()

+--------+----------+-------+---------------+
|order_id|order_date|cust_id|   order_status|
+--------+----------+-------+---------------+
|       1|2013-07-25|  11599|         CLOSED|
|       2|2013-07-25|    256|PENDING_PAYMENT|
|       3|2013-07-25|  12111|       COMPLETE|
|       4|2013-07-25|   8827|         CLOSED|
|       5|2013-07-25|  11318|       COMPLETE|
|       6|2013-07-25|   7130|       COMPLETE|
|       7|2013-07-25|   4530|       COMPLETE|
|       8|2013-07-25|   2911|     PROCESSING|
|       9|2013-07-25|   5657|PENDING_PAYMENT|
|      10|2013-07-25|   5648|PENDING_PAYMENT|
|      11|2013-07-25|    918| PAYMENT_REVIEW|
|      12|2013-07-25|   1837|         CLOSED|
|      13|2013-07-25|   9149|PENDING_PAYMENT|
|      14|2013-07-25|   9842|     PROCESSING|
|      15|2013-07-25|   2568|       COMPLETE|
|      16|2013-07-25|   7276|PENDING_PAYMENT|
|      17|2013-07-25|   2667|       COMPLETE|
|      18|2013-07-25|   1205|         CLOSED|
|      19|2013-07-25|   9488|PENDI

In [15]:
spark.sql("create database if not exists sukum16_caching_demo")

""


In [ ]:
spark.catalog.setCurrentDatabase("sukum16_caching_demo")

In [ ]:
! hdfs dfs -rm -r /user/itv020649/warehouse/sukum16_caching_demo.db

In [42]:
raw_df.write.format("csv").saveAsTable("sukum16_caching_demo.sukum16_order2")

In [37]:
! hadoop fs -ls /user/itv020649/warehouse

Found 3 items
drwxr-xr-x   - itv020649 supergroup          0 2025-10-13 11:03 /user/itv020649/warehouse/sukum16_caching_demo.db
drwxr-xr-x   - itv020649 supergroup          0 2025-10-12 11:45 /user/itv020649/warehouse/sukum16_caching_demo_ext.db
drwxr-xr-x   - itv020649 supergroup          0 2025-08-07 12:30 /user/itv020649/warehouse/sukum16_retail.db


In [38]:
! hadoop fs -ls /user/itv020649/warehouse/sukum16_caching_demo.db

Found 1 items
drwxr-xr-x   - itv020649 supergroup          0 2025-10-13 11:04 /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order4


1.1gb 9blocks 9partition 9tasks-> 9 files

Managed table

In [43]:
spark.sql("describe extended sukum16_caching_demo.sukum16_order2").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|            order_id|              bigint|   null|
|          order_date|                date|   null|
|             cust_id|              bigint|   null|
|        order_status|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|sukum16_caching_demo|       |
|               Table|      sukum16_order2|       |
|               Owner|           itv020649|       |
|        Created Time|Mon Oct 13 11:23:...|       |
|         Last Access|Wed Dec 31 19:00:...|       |
|          Created By|         Spark 2.4.7|       |
|                Type|             MANAGED|       |
|            Provider|                 csv|       |
|    Table Properties|[transient_lastDd...|       |
|          Statistics|     840836625 bytes|       |
|           

In [44]:
spark.sql("select count(*) from sukum16_caching_demo.sukum16_order2")

count(1)
25831125


In [45]:
spark.sql("cache table sukum16_caching_demo.sukum16_order2")

""


In [46]:
spark.sql("select count(*) from sukum16_caching_demo.sukum16_order2")

count(1)
25831125


In [ ]:
#process local that means it hit cache
#cache is not lazy

In [47]:
#uncache
spark.sql("uncache table sukum16_caching_demo.sukum16_order2")

""


In [48]:
spark.sql("cache lazy table sukum16_caching_demo.sukum16_order2")

""


In [49]:
spark.sql("select count(distinct (order_status)) from sukum16_caching_demo.sukum16_order2")

count(DISTINCT order_status)
9


In [ ]:
spark.sql("select distinct (order_status) from sukum16_caching_demo.sukum16_order2")

In [50]:
! hadoop fs -du -h /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2

0       0        /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/_SUCCESS
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/part-00000-8aa4f434-d950-4a2c-b5b4-359473403bcd-c000.csv
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/part-00001-8aa4f434-d950-4a2c-b5b4-359473403bcd-c000.csv
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/part-00002-8aa4f434-d950-4a2c-b5b4-359473403bcd-c000.csv
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/part-00003-8aa4f434-d950-4a2c-b5b4-359473403bcd-c000.csv
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/part-00004-8aa4f434-d950-4a2c-b5b4-359473403bcd-c000.csv
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_order2/part-00005-8aa4f434-d950-4a2c-b5b4-359473403bcd-c000.csv
95.7 M  287.0 M  /user/itv020649/warehouse/sukum16_caching_demo.db/sukum16_

In [ ]:
#9 partition for 800mb data because 9 files each file is less than 128 mb

In [51]:
spark.sql("insert into sukum16_caching_demo.sukum16_order2 values (11111,'2025-05-29',2222,'BOOKED')")

""


a new file is created as we have inserted a record

what happen if i do distinct now, cache was invalidated, its refreshing the cache

Node local as it didnot his cache

In [52]:
spark.sql("clear cache")
#uncache everything in this session

""


### Spark Catalog, Managed & External Tables

In [ ]:
spark.catalog.currentDatabase()

In [53]:
spark.sql("use sukum16_caching_demo")

""


In [54]:
spark.catalog.isCached("sukum16_caching_demo.sukum16_order2")

False

In [56]:
#external table
spark.sql("create database sukum16_caching_demo_ext")

""


In [57]:
spark.sql("create table sukum16_caching_demo_ext.sukum16_order_ext(order_id long,order_date date,cust_id long,order_status string) using csv location '/user/itv020649/'")  

""


In [58]:
! hadoop fs -cat /user/itv020649/orders_sukum16.csv

44,8602,37.19
35,5368,65.89
2,3391,40.64
47,6694,14.98
29,680,13.08
91,8900,24.59
70,3959,68.68
85,1733,28.53
53,9900,83.55
14,1505,4.32


In [59]:
spark.sql("show tables in sukum16_caching_demo_ext")

database,tableName,isTemporary
sukum16_caching_d...,sukum16_order_ext,false


In [60]:
spark.sql("describe extended sukum16_caching_demo_ext.sukum16_order_ext").show(30,False)

+----------------------------+---------------------------------------------------------+-------+
|col_name                    |data_type                                                |comment|
+----------------------------+---------------------------------------------------------+-------+
|order_id                    |bigint                                                   |null   |
|order_date                  |date                                                     |null   |
|cust_id                     |bigint                                                   |null   |
|order_status                |string                                                   |null   |
|                            |                                                         |       |
|# Detailed Table Information|                                                         |       |
|Database                    |sukum16_caching_demo_ext                                 |       |
|Table                       |

In [61]:
spark.sql("select * from sukum16_caching_demo_ext.sukum16_order_ext").show()

+--------+----------+-------+------------+
|order_id|order_date|cust_id|order_status|
+--------+----------+-------+------------+
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|    null|      null|   null|        null|
|   11111|2023-05-29|   2222|      BOOKED|
|   11111|2023-05-29|   2222|      BOOKED|
+--------+----------+-------+------------+



In [62]:
spark.sql("cache table sukum16_caching_demo_ext.sukum16_order_ext ")

""


In [63]:
#if you inser a row a new file is created
spark.sql("insert into sukum16_caching_demo_ext.sukum16_order_ext values (11111, '2023-05-29', 2222,'BOOKED')")

""


In [64]:
spark.sql("select count(*) from sukum16_caching_demo_ext.sukum16_order_ext")

count(1)
13


In [ ]:
! hadoop fs -cat  /user/itv020649/part-00000-a32ce55a-1b85-43cd-85c9-38a0b4a2f80c-c000.csv

In [65]:
spark.sql("select count(*) from sukum16_caching_demo_ext.sukum16_order_ext")

count(1)
13


In [66]:
#if a incremental file is put
#we need to refresh table
spark.catalog.refreshTable("sukum16_caching_demo_ext.sukum16_order_ext")

In [ ]:
spark.sql("refresh table sukum16_caching_demo_ext.sukum16_order_ext")

when the incremental file is deleated

In [ ]:
spark.sql("refresh table sukum16_caching_demo_ext.sukum16_order_ext")
#need to refresh table

### Cache Performance

can cacheing result lower performance

backend data -> data frame -> cache result

change.       with refresh DF change      2 refresh required for cache


if backed data is deleted we need to refresh

In [12]:
order_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [13]:
order_df.show()

+--------+----------+-------+---------------+
|order_id|order_date|cust_id|   order_status|
+--------+----------+-------+---------------+
|       1|2013-07-25|  11599|         CLOSED|
|       2|2013-07-25|    256|PENDING_PAYMENT|
|       3|2013-07-25|  12111|       COMPLETE|
|       4|2013-07-25|   8827|         CLOSED|
|       5|2013-07-25|  11318|       COMPLETE|
|       6|2013-07-25|   7130|       COMPLETE|
|       7|2013-07-25|   4530|       COMPLETE|
|       8|2013-07-25|   2911|     PROCESSING|
|       9|2013-07-25|   5657|PENDING_PAYMENT|
|      10|2013-07-25|   5648|PENDING_PAYMENT|
|      11|2013-07-25|    918| PAYMENT_REVIEW|
|      12|2013-07-25|   1837|         CLOSED|
|      13|2013-07-25|   9149|PENDING_PAYMENT|
|      14|2013-07-25|   9842|     PROCESSING|
|      15|2013-07-25|   2568|       COMPLETE|
|      16|2013-07-25|   7276|PENDING_PAYMENT|
|      17|2013-07-25|   2667|       COMPLETE|
|      18|2013-07-25|   1205|         CLOSED|
|      19|2013-07-25|   9488|PENDI

In [16]:
order_df.write.mode("overwrite").saveAsTable("sukum16_caching_demo.sukum16_order4")

parquet is default format

2 kinds of file format

row based -csv

column based -parquet,orc -works fast, meta data is also there

In [17]:
spark.sql("select count(*) from sukum16_caching_demo.sukum16_order4")

count(1)
25831125


less data is scanned as only subjected data is scanned

In [18]:
spark.sql("cache table sukum16_caching_demo.sukum16_order4")

""


In [20]:
spark.sql("select count(*) from sukum16_caching_demo.sukum16_order4")
#in case of cache its taking more time as it has to go through the whoole data as it going through data it cant check meta data

count(1)
25831125


parquet meta data:
    
    Column statistics record min/max, null_count, and optionally distinct_count, along with flags like min_is_exact and max_is_exact to qualify ordering semantics for correct pruning

### Persist

cache - dataframe/spark tables (memory and disk)

Persist - dataframe/spark tables (memory and disk) option parameter to set loaction memory or disk

5 argument:

1. disk

2. memory

3. off heap

4. deserilaised

5. number of replica

In [67]:
from pyspark.storagelevel import StorageLevel

In [71]:
#order_df.persist(storageLevel(Ture,True,False,False,1) - its default
order_df.persist(StorageLevel(True,False,False,False,1)) 
#lazy

order_id,order_date,cust_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [72]:
order_df.count()
#now caching

25831125

In [73]:
order_df.count()
#hit cache

25831125

In [74]:
order_df.persist(StorageLevel(True,True,False,True,1)) 


order_id,order_date,cust_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [75]:
order_df.count()


25831125

In [76]:
order_df.unpersist()

order_id,order_date,cust_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


node with 64 GB ram ,16 cpu core

3 executor- 20 gb,5 cpu core
60 gb

4 GB -  this is off heap memory